---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [78]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [51]:
allHomes = pd.read_csv('City_Zhvi_AllHomes.csv')


# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

allHomes['State'] = allHomes['State'].map(states)

In [82]:
def get_list_of_university_towns():

    uniTowns = pd.read_fwf('university_towns.txt', names=['RegionName']) #read txt file
    
    
    uniTowns['RegionName'] = uniTowns['RegionName'].str.replace('\(.*',"") # removing ()
    uniTowns['RegionName'] = uniTowns['RegionName'].replace(r'\\n',' ', regex=True) 
    uniTowns['RegionName'] = uniTowns['RegionName'].str.strip() #removing spaces
    
    
    
    df = pd.DataFrame(columns=['State', 'RegionName']) #create empty dataframe
    i=0
    sstr = ""
    rstr = " "
    
    for line in uniTowns['RegionName']:
        if 'edit' in line:  #if state
            sstr = line
        else:               #region
            rstr = line
            df.loc[i] = [sstr, rstr] #input state and region in newdf
            i += 1
      
    df['RegionName'] = df['RegionName'].str.replace('\[.*\]',"") # removing []
    df['State'] = df['State'].str.replace('\[.*\]',"") # removing []

    return df

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [57]:
def get_recession_start():
  
    GDP = ''
    i=0
    j=0
    gdp = pd.read_excel('gdplev.xls', skiprows = 219)
    
    gdp.columns = ['Unnamed0', 'Unnamed1','Unnamed2','Unnamed3','dateQ','Current','g2009','Unnamed7'] #column names
                
    for i in range(len(gdp)):
        val0 = gdp.iloc[i, 6]  # 6 = gdp2009
        i += 1
        val1 = gdp.iloc[i, 6]
        j = i     #1st year of fall
        i += 1
        val2 = gdp.iloc[i, 6]
        i += 1
        i -= 3
        if val0 <= val1:
            i += 1
        else:
            if val1 > val2: # fall in gdp
                GDP = gdp.iloc[j, 4]  # 4 = dateQ
                break

    return GDP


In [53]:
def get_recession_end():
   
    GDP = ''
    i=0
    j=0
    gdp = pd.read_excel('gdplev.xls', skiprows = 219)
    
    gdp.columns = ['Unnamed0', 'Unnamed1','Unnamed2','Unnamed3','dateQ','Current','g2009','Unnamed7']
                
    
    start = get_recession_start() #getting fall in gdp to calculate the next rise

    for i in range(len(gdp)):
        if gdp.iloc[i,4] <= start:  # while fall is not encountered
            i += 1
        else:
            val0 = gdp.iloc[i, 6]
            i += 1
            val1 = gdp.iloc[i, 6]
            i += 1
            val2 = gdp.iloc[i, 6]
            j = i       # 2nd year of rise
            i += 1
            i -= 3
            if val0 >= val1:
                i += 1
            else:
                if val1 < val2:  #rise in gdp
                    GDP = gdp.iloc[j, 4]
                    break

    return GDP


In [54]:
def get_recession_bottom():

    start = get_recession_start()
    end = get_recession_end()
    
    gdp = pd.read_excel('gdplev.xls', skiprows = 219)
    
    gdp.columns = ['Unnamed0', 'Unnamed1','Unnamed2','Unnamed3','dateQ','Current','g2009','Unnamed7']
    
    i = 0
    minimum = 99999
    bottom = ''
    
    for i in range(len(gdp)):
        if gdp.iloc[i,4] <= start:
            i += 1
        else:
            while gdp.iloc[i,4] < end :
                if gdp.iloc[i, 6] <= minimum:
                    minimum = gdp.iloc[i,6]
                    bottom = gdp.iloc[i,4]
                i += 1             
            
      
    return bottom


In [55]:
def convert_housing_data_to_quarters():
  
    allHomes = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    allHomes = allHomes.drop(allHomes.iloc[:, 6:51], axis=1)
  
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    allHomes['State'] = allHomes['State'].map(states)
    
    
    df = pd.DataFrame(columns=['State','RegionName'])
    
    df = pd.concat([allHomes.ix[:,i:i+3].mean(axis=1) for i in range(6,len(allHomes.columns),3)], axis=1)
    
        
    df['State'] = allHomes['State']
    df['RegionName'] = allHomes['RegionName']
    
    df = df.set_index(['State','RegionName'])
    df.columns=['2000q1', '2000q2','2000q3','2000q4','2001q1','2001q2','2001q3','2001q4','2002q1','2002q2','2002q3','2002q4',
                '2003q1','2003q2','2003q3','2003q4','2004q1','2004q2','2004q3','2004q4','2005q1','2005q2','2005q3','2005q4',
                '2006q1','2006q2','2006q3','2006q4','2007q1','2007q2','2007q3','2007q4','2008q1','2008q2','2008q3','2008q4',
                '2009q1','2009q2','2009q3','2009q4','2010q1','2010q2','2010q3','2010q4','2011q1','2011q2','2011q3','2011q4',
                '2012q1','2012q2','2012q3','2012q4','2013q1','2013q2','2013q3','2013q4','2014q1','2014q2','2014q3','2014q4',
                '2015q1','2015q2','2015q3','2015q4','2016q1','2016q2','2016q3']

    return df.sort_index()




In [83]:
def run_ttest():
 
    uniTowns = get_list_of_university_towns()
    start = get_recession_start()
    bottom = get_recession_bottom()
    houses = convert_housing_data_to_quarters()
    
    houses = houses.reset_index()
    houses['recession_diff'] = houses[start] - houses[bottom] #find diff
    
    townhouse = pd.merge(houses, uniTowns, how='inner', on=['State', 'RegionName'])
    townhouse['isutown'] = True   #create a column which tells whether it is a uniTown
    
    #merging both the unitown data frame and total towns dataframe 
    houses = pd.merge(houses, townhouse, how='outer', on = ['State', 'RegionName', bottom, start, 'recession_diff'])
                                                            
    houses['isutown'] = houses['isutown'].fillna(False)  #if not in townhouse(uni towns) df
                                                            
    unitowns = houses[houses['isutown'] == True]
                                                            
    not_unitowns = houses[houses['isutown'] == False]
    
    #hypothesis testing
    t, p = ttest_ind(unitowns['recession_diff'].dropna(), not_unitowns['recession_diff'].dropna()) 
                      
    different = True if p < 0.01 else False  #True = there is difference
                      
    if unitowns['recession_diff'].mean() < not_unitowns['recession_diff'].mean():
        better = "university town" 
    else:
        better = "non-university town"
    
    return different,p,better

run_ttest()

(True, 0.0043252148535112009, 'university town')